In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:

env = Env.STAGING

### Agility Parity LONG CODE

In [5]:
agility_config = {'detection_window': (0.07, 0.750), 'countdown': 5, 'proportion_correct_balanced': True}
readiness_config = {'detection_window': (0.1, 1.0), 'countdown': 5, 'retained_reaction_time_count': (1,5), 'minimum_reaction_time_count': 2}
focus_config= {'detection_window': (0.08, 1.0), 'countdown': 5, 'retained_reaction_time_count': (0,90), 'minimum_reaction_time_count': 2, 'lapse_multiplier':1}

In [35]:


start_date = dt.datetime(2024, 1, 1, 0, 0, 0)
end_date = dt.datetime(2024, 12, 30, 0, 0, 0)

### Creating the Final Dataframe to BigQuery

In [58]:



user_df = get_users(env)
user_df = user_df.drop(columns='created_at')
pison_users_df = pd.read_csv('pison_users.csv')
env = Env.STAGING
algo_calculations_df = []

### CHANGE TO [None, 'READY', 'AGILITY', or 'FOCUS']
test_type = None



users_df = get_specific_users(user_df, pison_users_df)


big_df = create_test_df(env, users_df, algo_calculations_df, test_type, start_date, end_date)


Checking parity...


Checking parity...
                                         id reaction_test_type  \
0      2f42cdfc-0814-5d4e-8cc4-38959e15313e              READY   
1      816b9a72-74a8-551a-a6e2-c50052c7556d              READY   
2      54f88a07-3a67-5836-a328-0d773b2f5690              READY   
3      686f0cfe-5a8c-55d4-8ff6-30b9e7573cd8              READY   
4      5c2e67be-67ed-55b1-a36d-dc714604c93a            AGILITY   
...                                     ...                ...   
11692  5ea24acc-42a8-447d-b07f-fa098b3e245e              READY   
11693  d6acd8c8-bf06-4e27-97e9-fce04a929ee8              READY   
11694  4070ffe4-6795-4ab6-83f2-7069c059e950              READY   
11695  1179c993-410d-4b82-b9d0-c58c3ce71c8e              READY   
11696  2401c81d-163c-40b6-acd0-de4861ed66e2              READY   

                            user_id                       created_at  \
0      pvSUbI3qLsXDg3a7hfrHP9F3kk22 2024-08-20 15:15:56.730000+00:00   
1      pvSUbI3qLsXDg3a7hfrHP9F3kk22 2024-08-

### Incorporates Team DF

In [59]:


# Load the team DataFrame and rename the 'Team' column
team_df = pd.read_csv('pison_users.csv')
team_df = team_df[['email', 'Team']]
team_df.rename(columns={'Team': 'pison_team'}, inplace=True)
team_df['email'] = team_df['email'].astype(str).str.split(',')

# Explode the list into separate rows
team_df_exploded = team_df.explode('email')

# Strip any leading/trailing whitespace from email addresses
team_df_exploded['email'] = team_df_exploded['email'].str.strip()

# Mapping from email to pison_team
email_to_team_map = team_df_exploded.set_index('email')['pison_team'].to_dict()

big_df['pison_team'] = big_df['email'].map(email_to_team_map)

### BigQuery

In [61]:



project_id = 'core-aca65d38'
dataset_name = 'Big_Tables'

focus_table = 'Focus_Table'
agility_table = 'Agility_Table'
ready_table = 'Ready_Table'
super_table = 'Super_Table'



CHOOSE_YOUR_DESTINATION_TABLE = super_table   # CHOOSE YOUR TABLE DESTINATION HERE

destination_table = f'{project_id}.{dataset_name}.{CHOOSE_YOUR_DESTINATION_TABLE}'
rel_cred_path = "key.json"  # Adjust as per your directory structure
cred_path = os.path.abspath(rel_cred_path)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = cred_path
credentials = service_account.Credentials.from_service_account_file(cred_path)

try:
    to_gbq(
        big_df, # CHANGE TO YOUR DF THAT YOU WANT TO PUT
        destination_table,
        project_id=project_id,
        if_exists='replace',
        credentials=credentials
    )
    print("Data successfully written to BigQuery!")
except Exception as e:
    print(f"Error writing to BigQuery: {str(e)}")



100% 1/1 [00:00<00:00, 6168.09it/s]

Data successfully written to BigQuery!
